In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
from datetime import datetime
geolocator = Nominatim(user_agent="my_user_agent")

loc = geolocator.geocode("Alsace, France")
print(loc)

Alsace, France métropolitaine, France


In [2]:
# Import Meteostat library
from meteostat import Stations, Daily

# Get nearby weather stations
stations = Stations()
stations = stations.nearby(loc.latitude, loc.longitude)
station = stations.fetch(10)
station = station[station['distance'] <= 5e5]

In [3]:
station

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
LFGA0,Colmar / Houssen,FR,A,<NA>,LFGA,48.1099,7.3590,191.0,Europe/Paris,2020-01-14,2023-03-22,2020-01-14,2022-04-25,2020-01-01,2022-01-01,15898.343254
D5453,Weisweil-Waldeckhof,DE,BW,<NA>,<NA>,48.1880,7.6977,173.0,Europe/Berlin,2005-04-01,2023-03-20,1992-06-01,2022-04-25,1992-01-01,2022-01-01,22552.944382
D2388,Ihringen,DE,BW,<NA>,<NA>,48.0419,7.6322,193.0,Europe/Berlin,1990-06-01,2023-03-20,1990-06-01,2022-04-26,1990-01-01,2022-01-01,28446.807592
D1224,Emmendingen-Mundingen,DE,BW,<NA>,<NA>,48.1378,7.8351,201.0,Europe/Berlin,2002-12-01,2023-03-20,1975-11-01,2023-03-20,1975-01-01,2022-01-01,34031.233244
10805,Lahr,DE,BW,10805,EDTL,48.3667,7.8333,155.0,Europe/Berlin,1973-01-01,2023-03-22,1950-01-01,2023-03-20,1950-01-01,2022-01-01,34097.386360
07197,Colmar,FR,A,07197,LFSC,47.9167,7.4000,211.0,Europe/Paris,1973-01-01,2023-02-05,1986-03-12,2022-03-28,2005-01-01,2021-01-01,36973.858585
07190,Strasbourg,FR,A,07190,LFST,48.5500,7.6333,153.0,Europe/Paris,1931-01-01,2023-03-22,1923-05-01,2023-03-16,1923-01-01,2022-01-01,37374.607862
99854,Neuhof,FR,A,99854,EQAE,48.5500,7.7600,139.0,Europe/Paris,2001-11-08,2002-04-25,NaT,NaT,NaT,NaT,42382.080180
10803,Freiburg,DE,BW,10803,EDTF,48.0000,7.8500,269.0,Europe/Berlin,1949-01-01,2023-03-22,1874-01-01,2023-03-20,1874-01-01,2022-01-01,42971.621129


In [4]:
from datetime import datetime, timedelta
import numpy as np

# Set time period
start = datetime(2000, 1, 1)
end = datetime(2022, 12, 31)

# Get Daily data
data = Daily(station.index.values[0], start, end)
data = data.fetch()
data = data.reset_index()
data = data[['time','tavg', 'tmin', 'tmax']]


In [41]:
#data['time']

start = datetime(2000, 1, 1)
end = datetime(2022, 12, 31)

dates = pd.date_range(start='2000-01-01', end='2022-12-31')



data = pd.DataFrame(columns=['time','tavg', 'tmin', 'tmax'], dtype='datetime64[ns]')
while (data['time'].empty) | (data['time'].min() > start) :
    if i <= len(station.index) - 1:
        next_data = Daily(station.index.values[i], start, end)
        next_data = next_data.fetch()
        next_data = next_data.reset_index()
        next_data = next_data[['time','tavg', 'tmin', 'tmax']]
        data = pd.concat([next_data, data])
        end = data['time'][0] - pd.Timedelta(days=1)
        i = i + 1
    else:
        break
if not dates.isin(data['time']).all():
    print(f'Some dates not recorded: {round(100*len(dates[~dates.isin(data["time"])])/len(dates),2)}%')

Some dates not recorded: 5.89%


In [46]:
next_data

,time,tavg,tmin,tmax
0,2000-01-01,4.6,2.0,5.9
1,2000-01-02,3.7,1.5,6.5
2,2000-01-03,1.6,-0.5,5.4
3,2000-01-04,7.3,-1.0,10.0
4,2000-01-05,6.0,3.4,11.0
...,...,...,...,...
7901,2022-12-27,4.7,2.2,7.8
7902,2022-12-28,7.7,3.3,10.8
7903,2022-12-29,11.0,8.5,13.0
7904,2022-12-30,9.0,5.8,12.4


In [4]:
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month

In [5]:
data_month_year = data.groupby(['year', 'month'])[['tavg', 'tmin', 'tmax']].mean().reset_index()

In [6]:
data_month = data_month_year.groupby('month')[['tavg', 'tmin', 'tmax']].mean().reset_index()
data_month.columns = ['month', 'tavg_month', 'tmin_month', 'tmax_month']
data_month_year = data_month_year.merge(data_month, how = 'left', on = 'month')

In [7]:
data_month_year['tflux_month'] = data_month_year['tavg'] - data_month_year['tavg_month']

In [57]:
data_month_year = pd.read_csv('data/france_regions_weather_data.csv')

In [58]:
convert_months = {
    1:"Jan",
    2:"Feb",
    3:"Mar",
    4:"Apr",
    5:"May",
    6:"Jun", 
    7:"July",
    8:"Aug",
    9:"Sep",
    10:"Oct",
    11:"Nov",
    12:"Dec"
}
data_month_year['month'] = data_month_year['month'].replace(convert_months)

In [62]:
data_month_year[(data_month_year['location'] == 'Alsace, France')]

,year,month,location,tavg,tmin,tmax,tavg_month,tmin_month,tmax_month
0,2020,Jan,"Alsace, France",4.411765,0.705882,8.058824,3.072739,-0.251803,6.333586
1,2020,Feb,"Alsace, France",7.903448,3.862069,12.413793,6.572578,2.382594,11.006979
2,2020,Mar,"Alsace, France",7.846667,3.200000,12.533333,7.562222,2.433333,12.664444
3,2020,Apr,"Alsace, France",14.539130,7.130435,21.391304,11.443043,5.503478,17.138213
4,2020,May,"Alsace, France",16.219048,10.190476,21.761905,15.567640,9.951664,20.888377
5,2020,Jun,"Alsace, France",18.740000,13.920000,23.520000,20.365556,15.063333,25.711111
6,2020,July,"Alsace, France",21.712000,15.280000,27.440000,21.396473,15.619140,26.791828
7,2020,Aug,"Alsace, France",22.904762,17.000000,28.666667,21.529544,16.192473,26.867384
8,2020,Sep,"Alsace, France",17.762963,12.148148,23.444444,17.114321,11.854938,22.477037
9,2020,Oct,"Alsace, France",11.708696,8.043478,15.347826,12.579243,8.052127,17.599813


In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plot_data = data_month_year[(data_month_year['year'] == 2006) & (data_month_year['location'] == 'Alsace, France')]

fig = make_subplots(specs=[[{"secondary_y": True}]])

#fig.add_trace(go.Bar(x = plot_data['month'], y = plot_data['tflux_month'], name = "2005"))
fig.update_layout(
    shapes = [
    {
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': 'Mar',
            'y0': 0,
            'x1': 'Oct',
            'y1': 1,
            'fillcolor': '#ffffe0',
            'opacity': 0.5,
            'line': {
                'width': 0,
            }
    
    }
    ]
)
fig.add_trace(go.Scatter(x = plot_data['month'], y = plot_data['tmax']))
fig.add_trace(go.Scatter(x = plot_data['month'], y = plot_data['tmin']))
fig.update_layout(
    plot_bgcolor='#FFFFFF',
        yaxis=dict(range=[0, 45])
)
fig.add_vline(
    x = 'Mar',
    line_dash = 'dash',
)
fig.add_annotation(
    x="Mar",
    y = 45,
    text="Growth Cycle Start",
    showarrow = False
)
fig.add_vline(
    x = 'Oct',
    line_dash = 'dash',
)
fig.add_annotation(
    x = 'Oct',
    y = 45,
    text = 'Harvest',
    showarrow=False
)

In [42]:
import pandas as pd 
pd.read_csv('data/france-wine-regions.csv')

,Place
0,"Alsace, France"
1,"Beaujolais, France"
2,"Bordeaux, France"
3,"Bugey Savoie, France"
4,"Burgundy, France"
5,"Chablis Chatillonais, France"
6,"Champagne, France"
7,"Corsica, France"
8,"Languedoc, France"
9,"Loire Valley, France"


In [50]:
import csv

# Open the CSV file and read it into a list
# Open the CSV file and read it into a list
with open('data/france-wine-regions.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    data = [row[0] for row in reader]
    data = data[1:] # drop the header

data

['Alsace, France',
 'Beaujolais, France',
 'Bordeaux, France',
 'Bugey Savoie, France',
 'Burgundy, France',
 'Chablis Chatillonais, France',
 'Champagne, France',
 'Corsica, France',
 'Languedoc, France',
 'Loire Valley, France',
 'Provence, France',
 'Rhône Valley, France']